In [4]:
import os, requests, sys, time

BASE = "https://api.balldontlie.io/v1"
API_KEY = os.getenv("BALLDONTLIE_API_KEY") or "5cb5a83d-6dbf-42fe-a323-6f716fb9506f"
HEADERS = {"Authorization": API_KEY}

def get_player_id(name: str, team_abbr: str | None = None, max_pages=3):
    # split name safely
    parts = [p for p in name.split() if p.strip()]
    first, last = (parts[0], parts[-1]) if parts else ("", "")

    per_page = 100
    cursor = None
    candidates = []

    # 1) try first_name + last_name (exact fields)
    for _ in range(max_pages):
        params = {"first_name": first, "last_name": last, "per_page": per_page}
        if cursor: params["cursor"] = cursor
        r = requests.get(f"{BASE}/players", params=params, headers=HEADERS, timeout=15)
        r.raise_for_status()
        payload = r.json()
        data = payload.get("data", [])
        candidates.extend(data)
        cursor = payload.get("meta", {}).get("next_cursor")
        if not cursor or not data:
            break
        time.sleep(0.1)

    # 2) fallback: search by last name then filter locally
    if not candidates:
        cursor = None
        for _ in range(max_pages):
            params = {"search": last or first, "per_page": per_page}
            if cursor: params["cursor"] = cursor
            r = requests.get(f"{BASE}/players", params=params, headers=HEADERS, timeout=15)
            r.raise_for_status()
            payload = r.json()
            data = payload.get("data", [])
            candidates.extend([p for p in data
                               if (not first or p.get("first_name","").lower()==first.lower())
                               and (not last  or p.get("last_name","").lower()==last.lower())])
            cursor = payload.get("meta", {}).get("next_cursor")
            if not cursor or not data:
                break
            time.sleep(0.1)

    if not candidates:
        raise ValueError(f"No players found for '{name}'")

    # optional team filter (e.g., 'POR', 'MIL')
    if team_abbr:
        pool = [p for p in candidates if (p.get("team") or {}).get("abbreviation")==team_abbr] or candidates
    else:
        pool = candidates

    p = pool[0]
    return p["id"], f"{p['first_name']} {p['last_name']}", (p.get("team") or {}).get("abbreviation")

def get_injuries(player_name: str, team_abbr: str | None = None,
                 per_page: int = 100, max_pages: int = 200) -> list[str]:
    """
    Return all injury 'description' strings for the given player from balldontlie.
    - player_name: e.g., 'Damian Lillard'
    - team_abbr:   optional disambiguator, e.g., 'MIL'
    """
    if not API_KEY:
        raise RuntimeError("BALLDONTLIE_API_KEY is not set")

    player_id, resolved_name, resolved_team = get_player_id(player_name, team_abbr)
    cursor = None
    descriptions: list[str] = []
    seen = set()  # dedupe while preserving order

    for _ in range(max_pages):
        params = {"player_ids[]": player_id, "per_page": per_page}
        if cursor: params["cursor"] = cursor
        r = requests.get(f"{BASE}/player_injuries", params=params, headers=HEADERS, timeout=20)
        if r.status_code == 401:
            raise RuntimeError("401 Unauthorized when calling /player_injuries — check API key / header.")
        if r.status_code == 429:
            # Gentle backoff if rate-limited
            time.sleep(0.5)
            continue
        r.raise_for_status()
        payload = r.json()
        data = payload.get("data", [])

        # Collect descriptions
        for item in data:
            desc = (item.get("description") or "").strip()
            print(desc)
            if desc and desc not in seen:
                seen.add(desc)
                descriptions.append(desc)

        cursor = payload.get("meta", {}).get("next_cursor")
        if not cursor or not data:
            break
        time.sleep(0.15)

    # Optional: most records include a 'return_date' like 'Nov 17' (no year).
    # If you want newest-first and the API returns them mixed, you can keep as-is
    # or implement custom ordering once they add richer timestamps.

    return descriptions

# Example:
print(get_injuries("Damian Lillard", "MIL"))


Sep 25: Lillard won't be back on the court for the 2025-26 season, as he made clear during an interview with YouTube influencer Speed on Wednesday.
["Sep 25: Lillard won't be back on the court for the 2025-26 season, as he made clear during an interview with YouTube influencer Speed on Wednesday."]
